import

In [1]:
# === Cell 1: Imports ===

import warnings
warnings.filterwarnings("ignore")

from sentence_transformers import SentenceTransformer
import hopsworks
import config


In [2]:
import os
# os.environ["HF_API_KEY"] = ""
os.environ["SILICONFLOW_API_KEY"] = "sk-rrswmweoulftfoccukdcuvyrlvbtblbufvhtvmrivwrcguaz"

In [3]:
# === Cell 2: Connect to Hopsworks ===

from config import HOPSWORKS_API_KEY
# project = hopsworks.login()

project = hopsworks.login(
        # project=HOPSWORKS_PROJECT,
        api_key_value=HOPSWORKS_API_KEY
    )
fs = project.get_feature_store()


2026-01-11 19:43:10,596 INFO: Initializing external client
2026-01-11 19:43:10,596 INFO: Base URL: https://c.app.hopsworks.ai:443
2026-01-11 19:43:12,264 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1286333


In [4]:
# === Cell 3: Load Feature Views ===

metadata_fv = fs.get_feature_view(
    name="paper_metadata_fv_2",
    version=2,
)

chunk_fv = fs.get_feature_view(
    name="paper_chunk_fv_2",
    version=2,
)

metadata_fv.init_serving(1)
chunk_fv.init_serving(1)


In [5]:
# === Cell 4: Load Embedding Model ===
sentence_transformer = SentenceTransformer(
    config.EMBEDDING_MODEL_NAME
)

2026-01-11 19:43:22,145 INFO: Use pytorch device_name: cpu
2026-01-11 19:43:22,145 INFO: Load pretrained SentenceTransformer: all-MiniLM-L6-v2


In [6]:
# === Cell 5: Similarity Search Engine ===

from functions.similarity_search import SimilaritySearchEngine

search_engine = SimilaritySearchEngine(
    embedding_model=sentence_transformer,
    metadata_feature_view=metadata_fv,
    chunk_feature_view=chunk_fv,
)


In [7]:
# === Cell 6: Context Builder ===

from functions.context_builder import ContextBuilder

context_builder = ContextBuilder(
    max_tokens=2000,
    max_chunks=8,
)


In [8]:
# === Cell 7: Prompt Synthesizer ===

# from functions.prompt_synthesis import PromptSynthesizer
# prompt_synthesizer = PromptSynthesizer()

from functions.prompt_synthesis_debug import DebugPromptSynthesizer

prompt_synthesizer = DebugPromptSynthesizer()



In [9]:
# === Cell 8: MCP Dispatcher ===

from functions.mcp_dispatcher import MCPDispatcher

mcp_dispatcher = MCPDispatcher(
    search_engine=search_engine
)


In [10]:
# === Cell 9: Agentic Inference ===

from functions.agent_loop import AgenticInference
from functions.llm_wrapper import LLMWrapper  

# llm = LLMWrapper(
#     model_name_or_path="mistralai/Mistral-7B-v0.1"  
# )# local llm
llm = LLMWrapper(
    model="Qwen/Qwen3-8B",
    base_url="http://api.siliconflow.cn/v1/",
    api_key=os.getenv("SILICONFLOW_API_KEY"),
    temperature=0.2,
    max_tokens=1024,
)

agent = AgenticInference(
    llm=llm,
    search_engine=search_engine,
    context_builder=context_builder,
    prompt_synthesizer=prompt_synthesizer,
    mcp_dispatcher=mcp_dispatcher,
)


In [11]:
# === Cell 10: Run Agent ===

from functions.agent_ui import launch_agent_ui

ui = launch_agent_ui(agent)
ui.launch(inline=True)


* Running on local URL:  http://127.0.0.1:7860
2026-01-11 19:43:27,714 INFO: HTTP Request: GET http://127.0.0.1:7860/gradio_api/startup-events "HTTP/1.1 200 OK"
2026-01-11 19:43:27,734 INFO: HTTP Request: HEAD http://127.0.0.1:7860/ "HTTP/1.1 200 OK"
* To create a public link, set `share=True` in `launch()`.


2026-01-11 19:43:28,182 INFO: HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
2026-01-11 19:43:44,742 WARNING: DeprecationWarning: 'HTTP_422_UNPROCESSABLE_ENTITY' is deprecated. Use 'HTTP_422_UNPROCESSABLE_CONTENT' instead.



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

PAPER METADATA: {'T55ZK2Y7': {'title': 'PCG Signal Acquisition and Classification for Heart Failure Detection: Recent Advances and Implementation of Memory-Efficient Classifiers for Edge Computing-Based Wearable Devices', 'abstract': 'Conventional diagnostic tools for cardiovascular diseases usually employ expensive instrumentation and require specialized medical staff. An inexpensive and non-invasive alternative is the phonocardiogram (PCG). This paper presents the development of classifiers for binary (Normal/Pathological) and multiclass classifications of PCG signals. The latter discerns between a subset of heart diseases (mitral valve prolapse (MVP), coronary disease (CAD), and benign murmurs (Benign)). Two balanced datasets were created from the Physionet 2016/CinC database, consisting of 10104 and 13136 5-s frames. A custom preprocessing chain includes denoising, normalizing, and splitting the PCG signals, making them suitable to extract the scalar features set, constituting the 